In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import HuberRegressor

# Diagram settings
title_font_size = 12
axis_font_size = 12

# For reproducibility
np.random.seed(42)

In [5]:
# Visualize loss functions: MSE vs. MAE vs. Huber
def loss_functions_visualize():    
    # ---------------------------------------------------
    # 1. Visualize MSE, MAE, and Huber Loss
    # ---------------------------------------------------
    errors = np.linspace(-2, 2, 200)
    delta = 1.0  # Huber threshold
    
    mse = errors**2
    mae = np.abs(errors)
    huber = np.where(np.abs(errors) <= delta,
                     0.5 * errors**2,
                     delta * (np.abs(errors) - 0.5 * delta))
    
    plt.figure(figsize=(7, 5))
    plt.plot(errors, mse, label="MSE", linewidth=2)
    plt.plot(errors, mae, label="MAE", linewidth=2)
    plt.plot(errors, huber, label="Huber (δ=1)", linewidth=2)
    plt.title("Loss Functions vs. Prediction Error", fontsize=title_font_size)
    plt.xlabel("Prediction Error (y - ŷ)", fontsize=axis_font_size)
    plt.ylabel("Loss", fontsize=axis_font_size)
    plt.legend()
    plt.grid(True)
    plt.show()
    

In [ ]:
from ipywidgets import interactive, FloatSlider, IntSlider, VBox, Label, Layout

def linear_reg_loss_demo(num_outliers=50):
    # ---------------------------------------------------
    # 1. Create a simple regression dataset
    # ---------------------------------------------------
    n_samples = 500
    X, y = make_regression(n_samples=n_samples, n_features=1, noise=15, random_state=42)

    # Add outliers controlled by sliders
    outlier_indices = np.random.choice(n_samples, num_outliers, replace=False)
    y[outlier_indices] += 200 * np.random.randn(num_outliers)

    # ---------------------------------------------------
    # 2. Train models with different loss functions
    # ---------------------------------------------------
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    ols = LinearRegression().fit(X_scaled, y)

    sgd_mae = SGDRegressor(loss='epsilon_insensitive', epsilon=0.0, 
                           max_iter=10000, tol=1e-3, random_state=42).fit(X_scaled, y)
    
    huber_reg = HuberRegressor(epsilon=1.0, max_iter=1000).fit(X_scaled, y)
    
    # Predictions
    x_line = np.linspace(X_scaled.min(), X_scaled.max(), 100).reshape(-1, 1)
    y_ols = ols.predict(x_line)
    y_mae = sgd_mae.predict(x_line)
    y_huber = huber_reg.predict(x_line)
    
    # ---------------------------------------------------
    # 3. Plot results
    # ---------------------------------------------------
    plt.figure(figsize=(8, 6))
    plt.scatter(X_scaled, y, color='gray', alpha=0.4, label="Data")
    plt.plot(x_line, y_ols, label="MSE", linewidth=2)
    plt.plot(x_line, y_mae, label="MAE", linewidth=2)
    plt.plot(x_line, y_huber, label="Huber", linewidth=2)
    plt.title("Effect of Different Loss Functions on Fit", fontsize=title_font_size)
    plt.xlabel("Standardized X", fontsize=axis_font_size)
    plt.ylabel("y", fontsize=axis_font_size)
    plt.legend()
    plt.grid(True)
    plt.show()


# ---------------------------------------------------
    # 4. Create clearly labeled, wide sliders inside a titled block
    # ---------------------------------------------------
def linear_reg_loss_functions_demo_interact():
    num_outliers_slider = IntSlider(
        value=50, min=0, max=500, step=5,
        description="Number of Outliers",
        style={'description_width': '150px'},
        layout=Layout(width='500px')
    )

    ui_box = VBox([
        Label(value="📊 Controls", layout=Layout(margin="0 0 0 0")),
    ])

    interactive_plot = interactive(
        linear_reg_loss_demo,
        num_outliers=num_outliers_slider
    )

    display(ui_box, interactive_plot)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression, SGDRegressor, HuberRegressor
from ipywidgets import interactive, FloatSlider, IntSlider, VBox, Label, Layout
from IPython.display import display

def polynomial_regression_loss_demo(num_outliers=50, poly_estimator_degree=3):
    # ---------------------------------------------------
    # 1. Create nonlinear dataset (quadratic + noise)
    # ---------------------------------------------------
    np.random.seed(42)
    n_samples = 400
    X = np.linspace(-3, 3, n_samples).reshape(-1, 1)
    y = 3 * X.squeeze()**3 - 3 * X.squeeze()**2 - 4 * X.squeeze() + np.random.randn(n_samples) * 2

    # Add outliers
    outlier_indices = np.random.choice(n_samples, num_outliers, replace=False)
    y[outlier_indices] += 30 * np.random.randn(num_outliers)
    
    # Polynomial features
    poly = PolynomialFeatures(degree=poly_estimator_degree, include_bias=False)
    X_poly = poly.fit_transform(X)
    
    # Normalize for fairness
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_poly)
    
    # ---------------------------------------------------
    # 2. Train models with different loss functions
    # ---------------------------------------------------
    ols = LinearRegression().fit(X_scaled, y)
    sgd_mae = SGDRegressor(loss='epsilon_insensitive', epsilon=0.0, 
                           max_iter=10000, tol=1e-3, random_state=42).fit(X_scaled, y)
    huber_reg = HuberRegressor(epsilon=1.0, max_iter=1000).fit(X_scaled, y)
    
    # ---------------------------------------------------
    # 3. Predict on dense grid for smooth curves
    # ---------------------------------------------------
    X_line = np.linspace(X.min(), X.max(), 200).reshape(-1, 1)
    X_line_poly = poly.transform(X_line)
    X_line_scaled = scaler.transform(X_line_poly)
    
    y_ols = ols.predict(X_line_scaled)
    y_mae = sgd_mae.predict(X_line_scaled)
    y_huber = huber_reg.predict(X_line_scaled)
    
    # ---------------------------------------------------
    # 4. Visualization
    # ---------------------------------------------------
    plt.figure(figsize=(8, 6))
    plt.scatter(X, y, color='gray', alpha=0.4, label="Data")
    plt.plot(X_line, y_ols, label="MSE", linewidth=2)
    plt.plot(X_line, y_mae, label="MAE", linewidth=2)
    plt.plot(X_line, y_huber, label="Huber", linewidth=2)
    
    plt.title("Polynomial Regression with Different Loss Functions", fontsize=16)
    plt.xlabel("X", fontsize=14)
    plt.ylabel("y", fontsize=14)
    plt.legend()
    plt.grid(True)
    plt.show()


# ---------------------------------------------------
# 5. Interactive control panel (same style as before)
# ---------------------------------------------------
def polynomial_regression_loss_demo_interact():
    num_outliers_slider = IntSlider(
        value=10, min=0, max=200, step=5,
        description="Number of Outliers",
        style={'description_width': '150px'},
        layout=Layout(width='500px')
    )

    poly_estimator_degree = IntSlider(
        value=3, min=1, max=6, step=1,
        description="Polyn. Estimator Degree",
        style={'description_width': '150px'},
        layout=Layout(width='500px')
    )

    ui_box = VBox([
        Label(value="📊 Controls", layout=Layout(margin="0 0 0 0")),
    ])

    interactive_plot = interactive(
        polynomial_regression_loss_demo,
        num_outliers=num_outliers_slider,
        poly_estimator_degree=poly_estimator_degree
    )

    display(ui_box, interactive_plot)
